In [19]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3
%load_ext autoreload
%autoreload 2
from gemini_prompts import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Config
- You need a .env file with GEMINI_API_KEY defined

In [35]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY_PAID"))

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-002",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

Check that it works

In [36]:
response = chat_session.send_message("Hello there, what is your name ")
print(response.text)

I don't have a name. I'm a large language model, an AI assistant.



Function to form prompt to give gemini

In [37]:
context = context_single_answer_v1 + context_local_dataset_v1
answer_format = answer_format_v1
field_to_query = field_to_query_v1

In [38]:
def form_prompt(query, local_data):
    prompt = f"""

    {context}
    {query}
    {answer_format_v1}

    Here are the results of my search for this firm.
    {local_data}
    """
    return prompt

### Open firm databases

In [39]:
conn = sqlite3.connect("firm_database_llm.db")
cursor = conn.cursor()


# Drop the table if you're starting from scratch
# cursor.execute('''
# DROP TABLE IF EXISTS firm_properties
#                ''')

table_name = "firm_properties_gemini_without_grounding_local_dataset_v1"
# You can create different tables for different prompt types, gemini model etc, if you rename the table
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               Firm_Name TEXT NOT NULL,
               Registered_Address TEXT,
               CEO TEXT,
               Establishment_Year INT,
               Number_Of_Employees INT,
               Revenue_Size INT,
               Website TEXT,
               NAICS_Code INT,
               SIC_Code INT,
               Status TEXT,
               Dissolvement_Year INT,
               Company_Type TEXT,
               Previous_Names TEXT, 
               Alternative_Names TEXT, 
               Key_Executive_Personnel TEXT
               )
               ''')


web_search_table_name = "firms_web_search_results"
webscraping_table_name = "firms_web_search_website_scrapings"
conn_web_results = sqlite3.connect("firms_web_search_results.db")
conn_websites = sqlite3.connect("firms_web_search_website_scrapings.db")
cursor_websearch = conn_web_results.cursor()
cursor_websites = conn_websites.cursor()

In [40]:
# Check existing tables in the database
existing_tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print(existing_tables)

[('sqlite_sequence',), ('firm_properties_gemini_without_grounding_local_dataset_v1',)]


In [41]:
# table_to_drop = "firm_properties_gemini_without_grounding_local_dataset_v1"
# cursor.execute(f"DROP TABLE IF EXISTS {table_to_drop} ")
# conn.commit()

Check how many entries are here

In [42]:
cursor.execute(f''' SELECT COUNT(*) FROM {table_name} ''').fetchall()

[(655,)]

Check how many firms have their website scrapings

In [43]:
cursor_websites.execute(f''' SELECT COUNT(*) FROM {webscraping_table_name}''').fetchall()

[(1315,)]

Get list of firms we'll loop through from the webscraping dataset

In [44]:
cursor_websites.execute(f''' SELECT id, Firm_Name FROM {webscraping_table_name} ''')
firm_list = cursor_websites.fetchall()

In [45]:
single_query_per_field = True

In [46]:
# TODO: ADD grounding parameter

TODO : Check how the lengths are distributed to determine which data to not pass to gemini, otherwise we go bankrupt

In [60]:
length_distribution = {field: [] for field in fields}

# Loop through each field and calculate the length of non-null entries
for field in fields:
    cursor_websites.execute(f"SELECT {field} FROM {webscraping_table_name}  WHERE {field} IS NOT NULL")
    rows = cursor_websites.fetchall()
    for row in rows:
        if row[0] is not None:
            length_distribution[field].append(len(str(row[0])))

# Print the distribution of lengths for each field
for field, lengths in length_distribution.items():
    print(f"Field: {field}")
    print(f"Number of entries: {len(lengths)}")
    print(f"Min length: {min(lengths) if lengths else 0}")
    print(f"Max length: {max(lengths) if lengths else 0}")
    print(f"Average length: {sum(lengths) / len(lengths) if lengths else 0:.2f}")
    print("-" * 40)


KeyboardInterrupt: 

Single query per field

In [58]:
if single_query_per_field:
    context = context_single_field_v1
    # TODO: This needs a different prompt to return a single answer
    for firm_data in firm_list[:650]:
        firm_id = firm_data[0]
        firm_name = firm_data[1]

        # Check if firm already exists, insert row if it doesnt
        cursor.execute(f"SELECT id FROM {table_name} WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
        firm_row = cursor.fetchone()
        if firm_row is None:
            print("Inserting new firm:, ", firm_name)
            cursor.execute(f"INSERT INTO {table_name} (Firm_Name) VALUES (?)", (firm_name,))
            firm_id = cursor.lastrowid  
        else:
            # Get the existing firm's id
            print("Found row for firm, ", firm_name, firm_id)
            firm_id = firm_row[0]
        
        # begin new chat session
        chat_session = model.start_chat(
            history=[
            ]
        )

        # Update queries with firm name
        updated_queries = {key: value.format(firm_name=firm_name) for key, value in field_to_query.items()}


        for field in fields:
            print("Debug now for ", firm_name, field)

            # 1. Check if the field value in the prdiction database is NULL
            cursor.execute(f"SELECT {field} FROM {table_name} WHERE id = ? AND Firm_Name = ? AND {field} IS NOT NULL", (firm_id,firm_name,))
            if cursor.fetchone() is not None:
                print(f"Field '{field}' already has data for firm '{firm_name}', skipping.")
                continue
        
            # 2.1Get the web search results for the firm
            cursor_websearch.execute(f"SELECT {field} FROM firms_web_search_results WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
            web_search_result = cursor_websearch.fetchone()
            if web_search_result is not None:
                web_search_result = json.loads(web_search_result[0]) # TODO check indexing here
            else:
                web_search_result = "No web search data available"
            
            # 2.2 Get the website scraping results for the firm
            cursor_websites.execute(f"SELECT {field} FROM firms_web_search_website_scrapings WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
            website_scraping_result = cursor_websites.fetchone()
            if website_scraping_result is not None and website_scraping_result[0] is not None:
                website_scraping_result = json.loads(website_scraping_result[0])
            else:
                website_scraping_result = "No website scraping data available"
        
            data_to_pass_llm = {
                f"Results of searching the web for {firm_name}": web_search_result,
                f"Scraped contents of top 5 websites for {firm_name}": website_scraping_result
            }

             # Generate the prompt
            prompt = form_prompt(updated_queries[field], data_to_pass_llm)

            prompt_limit = 4000000
            if len(prompt) > prompt_limit:
                print(f"Prompt for {firm_name} - {field} is too long. Skipping.")
                continue
        
            # break
            success = False
            retries = 0
            response = None
            while not success and retries < 1:
                try:
                    response = model.generate_content(prompt).text
                    # response = chat_session.send_message(prompt).text
                    # print("Success for ", firm_name, field)
                    success = True
                except Exception as e:
                    print("Gemini Exception: ", e)
                    time.sleep(60) # Sleep for 1 minute to reset quota
                    response = None
                    
            print(f"Response for {firm_name} - {field}: ", response)
            # Update database
            cursor.execute(f"UPDATE {table_name} SET {field} = ? WHERE Firm_Name = ? AND id = ?", (response, firm_name, firm_id))
            
            conn.commit()

Found row for firm,  "TRACTION POWER SYSTEMS, INC." 1
Debug now for  "TRACTION POWER SYSTEMS, INC." Registered_Address
Field 'Registered_Address' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Debug now for  "TRACTION POWER SYSTEMS, INC." CEO
Field 'CEO' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Debug now for  "TRACTION POWER SYSTEMS, INC." Establishment_Year
Field 'Establishment_Year' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Debug now for  "TRACTION POWER SYSTEMS, INC." Number_Of_Employees
Field 'Number_Of_Employees' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Debug now for  "TRACTION POWER SYSTEMS, INC." Revenue_Size
Field 'Revenue_Size' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Debug now for  "TRACTION POWER SYSTEMS, INC." Website
Field 'Website' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Debug now for  "TRACTION POWER SYSTE

KeyboardInterrupt: 

Single query per firm
- doenst work too well since there is a lot of content

In [50]:
if not single_query_per_field:
    context = context_single_answer_v1
    for firm_data in firm_list[:10]:
        firm_id = firm_data[0]
        firm_name = firm_data[1]

        # Check if firm already exists, insert row if it doesnt
        print("Debug -- now on firm id ", firm_id)

        cursor.execute(f"SELECT id FROM {table_name} WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
        firm_row = cursor.fetchone()
        if firm_row is None:
            print("Inserting new firm:, ", firm_name)
            cursor.execute(f"INSERT INTO {table_name} (Firm_Name) VALUES (?)", (firm_name,))
            firm_id = cursor.lastrowid  
        else:
            # Get the existing firm's id
            print("Found row for firm, ", firm_name , firm_id)
            firm_id = firm_row[0]


        # Identify if any field is missing for this firm with a single query
        cursor.execute(f"""
            SELECT id FROM {table_name}
            WHERE id = ? AND Firm_Name = ? AND ({' OR '.join([f"{field} IS NULL" for field in fields])})
        """, (firm_id, firm_name,))

        # Check if any fields are missing
        firm_row_with_missing_fields = cursor.fetchone()
        if firm_row_with_missing_fields is None:
            print(f"All fields are filled for {firm_name}, skipping.")
            continue

        # 2.1Get the web search results and scrapings for the firm

        # 2.1Get the web search results for the firm
        local_data_to_pass_llm = {}
        for field in fields:
            cursor_websearch.execute(f"SELECT {field} FROM firms_web_search_results WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
            web_search_result = cursor_websearch.fetchone()
            if web_search_result is not None:
                web_search_result = json.loads(web_search_result[0]) # TODO check indexing here
            else:
                web_search_result = "No web search data available"
            
            # 2.2 Get the website scraping results for the firm
            cursor_websites.execute(f"SELECT {field} FROM firms_web_search_website_scrapings WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
            website_scraping_result = cursor_websites.fetchone()
            if website_scraping_result is not None and website_scraping_result[0] is not None:
                website_scraping_result = json.loads(website_scraping_result[0])
            else:
                website_scraping_result = "No website scraping data available"
        
            local_data_field = {
                f"Results of searching the web for {firm_name} + {field} ": web_search_result,
                f"Scraped contents of top 5 websites for {firm_name} + {field}": website_scraping_result
            }

            local_data_to_pass_llm[field] = local_data_field


        # Update queries with firm name
        updated_queries = {key: value.format(firm_name=firm_name) for key, value in field_to_query.items()}

        # Form prompt and query gemini
        prompt = form_prompt(updated_queries, local_data_to_pass_llm)

        # Send the prompt to model, try 3 times to get a response
        response = None 
        try:
            response = model.generate_content(prompt).text
            # print("Success for ", firm_name, field)
            success = True
        except Exception as e:
            print("Gemini Exception occured: ", e)

        print(f"Gemini response for {firm_name}: ", response)

        if not response:
            print("Gemini failed to respond, skipping")
            continue

        answers_per_field = response.split("*-*")
        answers_per_field = answers_per_field[:len(fields)]
        print(len(answers_per_field))

        if len(answers_per_field) != len(fields):
            print("Error: Number of answers do not match number of fields," , len(answers_per_field), len(fields))
            continue
    
        for field, answer in zip(fields, answers_per_field):

            print(f"Gemini response for {firm_name} - {field}: ", answer)
        
            cursor.execute(f"UPDATE {table_name} SET {field} = ? WHERE Firm_Name = ? AND id = ?", (answer, firm_name, firm_id))
            conn.commit()

    cursor.close()
    conn.close()
    cursor_websearch.close()
    conn_web_results.close()